# 0. Importing Libraries

In [31]:
import numpy as np
import pandas as pd
import requests

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Ex. 1 Worldbank Indicator API

## 1.1 Define the function

In [32]:
def query_worldbank(indicators, countries, years):
    '''
    Purpose
    -------
    To query data from the World Bank Indicator API for the specified parameters.

    Parameters
    ----------
    indicators (list) : A list of indicator codes to query.
    countries (list) : A list of country codes to query.
    years (list) : A list of years to query.

    Returns
    -------
    A pandas DataFrame containing the queried data, with countries as rows, indicators as columns, and years as the index.
    
    '''

    base_url = "http://api.worldbank.org/v2/country/"
    all_data = []
    api_call_count = 0  #... initializing counter for API calls

    for country in countries:
        for year in years:
            for indicator in indicators:
                page = 1
                while True:
                    url = f'{base_url}{country}/indicator/{indicator}?date={year}&format=json&page={page}'

                    try:
                        response = requests.get(url)
                        response.raise_for_status()
                        data = response.json()

                        api_call_count += 1  #... incrementing the API call counter

                        # Print the response for debugging
                        print("API Response:", data)

                        # Check if we have data
                        if len(data) < 2 or len(data[1]) == 0:
                            print(f'No data found for {indicator} in {country} for {year}.')
                            break

                        # Extract relevant data
                        for entry in data[1]:
                            entry_data = {
                                'year': entry['date'],
                                'countryiso3code': entry['countryiso3code'],
                                'country': entry['country']['value'],
                                'indicator': entry['indicator']['value'],
                                'value': entry['value']
                            }
                            all_data.append(entry_data)

                        # Check for pagination
                        pages = data[0]['pages']
                        if page >= pages:  #... checking if we are on the last page
                            break

                        page += 1  #... moving to the next page

                    # Error handling
                    except Exception as e:
                        print(f'Error for {indicator} in {country} for {year}: {e}')
                        break

    # Create DataFrame
    worldbank_df = pd.DataFrame(all_data)

    # Pivot the df to have separate columns for each indicator
    pivot_df = worldbank_df.pivot_table(
        index=['year', 'countryiso3code', 'country'],
        columns='indicator',
        values='value',
        aggfunc='first'
    ).reset_index()

    # Sort the df by year and countryiso3code
    pivot_df['year'] = pd.to_numeric(pivot_df['year'])  #... converting year to numeric for sorting
    pivot_df.sort_values(by=['year', 'countryiso3code'], inplace=True)  #... sorting by year
    pivot_df.rename_axis(None, axis=1, inplace=True)   #... resetting index for proper data management
    
    print(f'Total API calls: {api_call_count}')  #... printing total API calls
    return pivot_df


## 1.2 Demonstrating the function works

### 1.2.a Total Population of Germany and France (2015-2020)

In [33]:
# Execute function for task 1a and store it in a DataFrame
wb_df_1a = query_worldbank(['SP.POP.TOTL'], ['DEU','FRA'], [2015, 2016, 2017, 2018, 2019, 2020])
wb_df_1a

API Response: [{'page': 1, 'pages': 1, 'per_page': 50, 'total': 1, 'sourceid': '2', 'lastupdated': '2024-10-24'}, [{'indicator': {'id': 'SP.POP.TOTL', 'value': 'Population, total'}, 'country': {'id': 'DE', 'value': 'Germany'}, 'countryiso3code': 'DEU', 'date': '2015', 'value': 81686611, 'unit': '', 'obs_status': '', 'decimal': 0}]]
API Response: [{'page': 1, 'pages': 1, 'per_page': 50, 'total': 1, 'sourceid': '2', 'lastupdated': '2024-10-24'}, [{'indicator': {'id': 'SP.POP.TOTL', 'value': 'Population, total'}, 'country': {'id': 'DE', 'value': 'Germany'}, 'countryiso3code': 'DEU', 'date': '2016', 'value': 82348669, 'unit': '', 'obs_status': '', 'decimal': 0}]]
API Response: [{'page': 1, 'pages': 1, 'per_page': 50, 'total': 1, 'sourceid': '2', 'lastupdated': '2024-10-24'}, [{'indicator': {'id': 'SP.POP.TOTL', 'value': 'Population, total'}, 'country': {'id': 'DE', 'value': 'Germany'}, 'countryiso3code': 'DEU', 'date': '2017', 'value': 82657002, 'unit': '', 'obs_status': '', 'decimal': 0}]

,year,countryiso3code,country,"Population, total"
0,2015,DEU,Germany,81686611
1,2015,FRA,France,66548272
2,2016,DEU,Germany,82348669
3,2016,FRA,France,66724104
4,2017,DEU,Germany,82657002
5,2017,FRA,France,66918020
6,2018,DEU,Germany,82905782
7,2018,FRA,France,67158348
8,2019,DEU,Germany,83092962
9,2019,FRA,France,67388001


### 1.2.b Total Population, GDP and Life Expectancy for all countries (2012)

In [34]:
# Execute function for task 1b and store it in a DataFrame
wb_df_1b = query_worldbank(['SP.POP.TOTL', 'NY.GDP.MKTP.CD', 'SP.DYN.LE00.IN'], ['all'], [2012])
wb_df_1b.shape  #... displaying the shape

API Response: [{'page': 1, 'pages': 6, 'per_page': 50, 'total': 266, 'sourceid': '2', 'lastupdated': '2024-10-24'}, [{'indicator': {'id': 'SP.POP.TOTL', 'value': 'Population, total'}, 'country': {'id': 'ZH', 'value': 'Africa Eastern and Southern'}, 'countryiso3code': 'AFE', 'date': '2012', 'value': 552530654, 'unit': '', 'obs_status': '', 'decimal': 0}, {'indicator': {'id': 'SP.POP.TOTL', 'value': 'Population, total'}, 'country': {'id': 'ZI', 'value': 'Africa Western and Central'}, 'countryiso3code': 'AFW', 'date': '2012', 'value': 376797999, 'unit': '', 'obs_status': '', 'decimal': 0}, {'indicator': {'id': 'SP.POP.TOTL', 'value': 'Population, total'}, 'country': {'id': '1A', 'value': 'Arab World'}, 'countryiso3code': 'ARB', 'date': '2012', 'value': 380383408, 'unit': '', 'obs_status': '', 'decimal': 0}, {'indicator': {'id': 'SP.POP.TOTL', 'value': 'Population, total'}, 'country': {'id': 'S3', 'value': 'Caribbean small states'}, 'countryiso3code': 'CSS', 'date': '2012', 'value': 289880

(265, 6)

In [35]:
wb_df_1b.head(10)  #... displaying the first 10 rows

,year,countryiso3code,country,GDP (current US$),"Life expectancy at birth, total (years)","Population, total"
0,2012,,High income,5.171000e+13,79.043510,1.342744e+09
1,2012,,Low income,3.817016e+11,60.557900,5.400026e+08
2,2012,,Lower middle income,4.456528e+12,66.382162,2.610685e+09
3,2012,,Upper middle income,1.867495e+13,74.386075,2.618530e+09
4,2012,ABW,Aruba,2.615208e+09,75.531000,1.021120e+05
5,2012,AFE,Africa Eastern and Southern,9.526756e+11,60.050780,5.525307e+08
6,2012,AFG,Afghanistan,1.990733e+10,61.923000,3.046648e+07
7,2012,AFW,Africa Western and Central,7.377996e+11,55.340561,3.767980e+08
8,2012,AGO,Angola,1.280529e+11,58.623000,2.518829e+07
9,2012,ALB,Albania,1.231983e+10,78.064000,2.900401e+06


### 1.2.c State how many API calls your function makes for a) and b) respectively.

A.: As can be observed from above tasks, for task 1.a the API made 12 calls and for task 1.b the API made 18 calls

# Ex. 2 Medals Table + Worldbank df

## 2.a Creating a combined df

### 2.a.1 Prepping the medals table & the worldbank dataframe from task 1b

In [36]:
medals_df = pd.read_csv('data/medal_table.csv')  #... loading the medal table into a DataFrame
medals_df

,year,country,country_code,gold,silver,bronze
0,2012,United States,USA,46,28,30
1,2012,People's Republic of China,CHN,38,31,22
2,2012,Great Britain,GBR,29,17,19
3,2012,Russian Federation,RUS,20,20,27
4,2012,Republic of Korea,KOR,13,9,8
...,...,...,...,...,...,...
81,2012,Kingdom of Saudi Arabia,KSA,0,0,1
82,2012,Kuwait,KUW,0,0,1
83,2012,Morocco,MAR,0,0,1
84,2012,Tajikistan,TJK,0,0,1


renaming `countryiso3code` to `country_code` in `wb_df_1b` for ease of manipulation

In [37]:
wb_df_1b.rename(columns={'countryiso3code': 'country_code'}, inplace=True)
wb_df_1b

,year,country_code,country,GDP (current US$),"Life expectancy at birth, total (years)","Population, total"
0,2012,,High income,5.171000e+13,79.043510,1.342744e+09
1,2012,,Low income,3.817016e+11,60.557900,5.400026e+08
2,2012,,Lower middle income,4.456528e+12,66.382162,2.610685e+09
3,2012,,Upper middle income,1.867495e+13,74.386075,2.618530e+09
4,2012,ABW,Aruba,2.615208e+09,75.531000,1.021120e+05
...,...,...,...,...,...,...
260,2012,XKX,Kosovo,6.163484e+09,78.280000,1.807106e+06
261,2012,YEM,"Yemen, Rep.",3.540133e+10,67.343000,2.622339e+07
262,2012,ZAF,South Africa,4.344005e+11,61.846000,5.314503e+07
263,2012,ZMB,Zambia,2.550306e+10,58.867000,1.474466e+07


### 2.a.2 Reconciling the country names from both tables
* reconciling country names from both tables in order to account for maximum data

In [38]:
%pip install fuzzywuzzy python-Levenshtein  #... installing the necessary packages to match country names based on partial matches


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [39]:
from fuzzywuzzy import process, fuzz

In [40]:
wb_countries = wb_df_1b['country'].tolist()  #... country names from the World Bank df
medals_countries = medals_df['country'].tolist()  #... country names from the medals df

In [41]:
# Match country names from medals_df to those from wb_df_1b

def match_countries_unique(medals_countries, wb_countries, threshold=80):
    match_names_medals = []
    match_names_wb = []
    used_wb_names = set()  #... tracking used names
    unmatched_countries = []  #... tracking unmatched countries

    for medals_country in medals_countries:
        best_match, score = process.extractOne(
            medals_country, 
            wb_countries,
            scorer=fuzz.token_set_ratio
        )
        
        # Only add to the lists if score >= threshold and match is unique
        if score >= threshold and best_match not in used_wb_names:
            match_names_medals.append(medals_country)
            match_names_wb.append(best_match)
            used_wb_names.add(best_match)
        else:
            unmatched_countries.append(medals_country)

    # Create df for matched countries
    match_df = pd.DataFrame({
        'medals_country': match_names_medals, 
        'wb_country': match_names_wb
    })

    # Create df for unmatched countries
    unmatch_df = pd.DataFrame({
        'medals_country': unmatched_countries,
        'wb_country': [None] * len(unmatched_countries)  #... fill with None or NaN
    })

    # Combine the matched and unmatched dfs
    combined_df = pd.concat([match_df, unmatch_df], ignore_index=True)

    return combined_df

match_df = match_countries_unique(medals_countries, wb_countries)

In [42]:
pd.set_option('display.max_rows', None)  #... displaying all rows to allow for visual inspection of missing values
match_df

,medals_country,wb_country
0,United States,United States
1,People's Republic of China,China
2,Russian Federation,Russian Federation
3,Germany,Germany
4,France,France
5,Australia,Australia
6,Italy,Italy
7,Hungary,Hungary
8,Japan,Japan
9,Islamic Republic of Iran,"Iran, Islamic Rep."


In [43]:
# Account for the unmatched countries

# List to store matched country names
results = []

# Check for partial matches
keywords = ['Britain', 'Korea', 'Czech', 'Turkiye', 'Taipei', 'Taiwan', 'Slovak', 'Hong Kong', 'Viet nam']
for keyword in keywords:
    matches = wb_df_1b[wb_df_1b['country'].str.contains(keyword, case=False, na=False)]
    
    if not matches.empty:
        # Append matched country names to the results list
        country_names = matches['country'].tolist()
        results.append((keyword, country_names))
    else:
        # Append None for unmatched keywords
        results.append((keyword, None))

# Create a DataFrame from the results
possible_matches_df = pd.DataFrame(results, columns=['Keyword', 'Possible Matches'])

# Display the resulting DataFrame
possible_matches_df

,Keyword,Possible Matches
0,Britain,None
1,Korea,"[Korea, Rep., Korea, Dem. People's Rep.]"
2,Czech,[Czechia]
3,Turkiye,[Turkiye]
4,Taipei,None
5,Taiwan,None
6,Slovak,[Slovak Republic]
7,Hong Kong,"[Hong Kong SAR, China]"
8,Viet nam,[Viet Nam]


In [44]:
# Add additional matches to match_df

additional_matches = {
    'Great Britain': 'United Kingdom',
    'Republic of Korea': 'Korea, Rep.',
    'Democratic People\'s Republic of Korea': 'Korea, Dem. People\'s Rep.',
    'Czech Republic': 'Czechia',
    'Slovakia': 'Slovak Republic',
    'Turkey': 'Turkiye',
    'Hong Kong, China': 'Hong Kong SAR, China',
    'Vietnam': 'Viet Nam',
}

for medals_country, wb_country in additional_matches.items():
    match_df.loc[match_df['medals_country'] == medals_country, 'wb_country'] = wb_country

match_df

,medals_country,wb_country
0,United States,United States
1,People's Republic of China,China
2,Russian Federation,Russian Federation
3,Germany,Germany
4,France,France
5,Australia,Australia
6,Italy,Italy
7,Hungary,Hungary
8,Japan,Japan
9,Islamic Republic of Iran,"Iran, Islamic Rep."


In [45]:
# Add country codes

match_df = match_df.merge(
    medals_df[['country', 'country_code']],
    left_on='wb_country',
    right_on='country',
    how='left'
)

match_df.drop('country', axis=1, inplace=True)

match_df.rename(columns={'country_code': 'medals_country_code'}, inplace=True)

match_df

,medals_country,wb_country,medals_country_code
0,United States,United States,USA
1,People's Republic of China,China,NaN
2,Russian Federation,Russian Federation,RUS
3,Germany,Germany,GER
4,France,France,FRA
5,Australia,Australia,AUS
6,Italy,Italy,ITA
7,Hungary,Hungary,HUN
8,Japan,Japan,JPN
9,Islamic Republic of Iran,"Iran, Islamic Rep.",NaN


In [46]:
# Reorder the columns
match_df = match_df[['medals_country', 'medals_country_code', 'wb_country']]
match_df

,medals_country,medals_country_code,wb_country
0,United States,USA,United States
1,People's Republic of China,NaN,China
2,Russian Federation,RUS,Russian Federation
3,Germany,GER,Germany
4,France,FRA,France
5,Australia,AUS,Australia
6,Italy,ITA,Italy
7,Hungary,HUN,Hungary
8,Japan,JPN,Japan
9,Islamic Republic of Iran,NaN,"Iran, Islamic Rep."


In [47]:
# Look for missing country codes
empty_codes = match_df[match_df['medals_country_code'].isnull()]['medals_country'].tolist()
empty_codes

["People's Republic of China",
 'Islamic Republic of Iran',
 'The Bahamas',
 'Venezuela',
 'Egypt',
 'Kingdom of Saudi Arabia',
 'Great Britain',
 'Republic of Korea',
 'Czech Republic',
 "Democratic People's Republic of Korea",
 'Turkey',
 'Chinese Taipei',
 'Slovakia',
 'Hong Kong, China',
 'Vietnam']

In [48]:
# Add missing country codes

missing_country_codes = {
 'People\'s Republic of China':'CHN',
 'Islamic Republic of Iran':'IRI',
 'The Bahamas':'BAH',
 'Venezuela':'VEN',
 'Egypt':'EGY',
 'Kingdom of Saudi Arabia':'KSA',
 'Great Britain':'GBR',
 'Republic of Korea':'KOR',
 'Czech Republic':'CZE',
 "Democratic People's Republic of Korea":'PRK',
 'Turkey':'TUR',
 'Chinese Taipei':'TPE',
 'Slovakia':'SVK',
 'Hong Kong, China':'HKG',
 'Vietnam':'VIE',
}

for medals_country, medals_country_code in missing_country_codes.items():
    match_df.loc[match_df['medals_country'] == medals_country, 'medals_country_code'] = medals_country_code

match_df

,medals_country,medals_country_code,wb_country
0,United States,USA,United States
1,People's Republic of China,CHN,China
2,Russian Federation,RUS,Russian Federation
3,Germany,GER,Germany
4,France,FRA,France
5,Australia,AUS,Australia
6,Italy,ITA,Italy
7,Hungary,HUN,Hungary
8,Japan,JPN,Japan
9,Islamic Republic of Iran,IRI,"Iran, Islamic Rep."


In [49]:
# Import the medals columns

medals_cols = ['country_code', 'year', 'gold', 'silver', 'bronze']
medals_subset = medals_df[medals_cols]

match_df = match_df.merge(medals_subset, left_on='medals_country_code', right_on='country_code', how='left')

match_df.drop(columns=['country_code'], axis=1, inplace=True)

match_df

,medals_country,medals_country_code,wb_country,year,gold,silver,bronze
0,United States,USA,United States,2012,46,28,30
1,People's Republic of China,CHN,China,2012,38,31,22
2,Russian Federation,RUS,Russian Federation,2012,20,20,27
3,Germany,GER,Germany,2012,11,20,13
4,France,FRA,France,2012,11,11,13
5,Australia,AUS,Australia,2012,8,15,12
6,Italy,ITA,Italy,2012,8,9,11
7,Hungary,HUN,Hungary,2012,8,4,6
8,Japan,JPN,Japan,2012,7,14,17
9,Islamic Republic of Iran,IRI,"Iran, Islamic Rep.",2012,7,5,1


In [50]:
# Reorder the columns
match_df = match_df[['year', 'medals_country', 'medals_country_code', 'wb_country', 'gold', 'silver', 'bronze']]
match_df

,year,medals_country,medals_country_code,wb_country,gold,silver,bronze
0,2012,United States,USA,United States,46,28,30
1,2012,People's Republic of China,CHN,China,38,31,22
2,2012,Russian Federation,RUS,Russian Federation,20,20,27
3,2012,Germany,GER,Germany,11,20,13
4,2012,France,FRA,France,11,11,13
5,2012,Australia,AUS,Australia,8,15,12
6,2012,Italy,ITA,Italy,8,9,11
7,2012,Hungary,HUN,Hungary,8,4,6
8,2012,Japan,JPN,Japan,7,14,17
9,2012,Islamic Republic of Iran,IRI,"Iran, Islamic Rep.",7,5,1


In [51]:
# Sort by the number of gold medals
match_sorted = match_df.sort_values(by='gold', ascending=False).reset_index(drop=True)
match_sorted 

,year,medals_country,medals_country_code,wb_country,gold,silver,bronze
0,2012,United States,USA,United States,46,28,30
1,2012,People's Republic of China,CHN,China,38,31,22
2,2012,Great Britain,GBR,United Kingdom,29,17,19
3,2012,Russian Federation,RUS,Russian Federation,20,20,27
4,2012,Republic of Korea,KOR,"Korea, Rep.",13,9,8
5,2012,Germany,GER,Germany,11,20,13
6,2012,France,FRA,France,11,11,13
7,2012,Hungary,HUN,Hungary,8,4,6
8,2012,Italy,ITA,Italy,8,9,11
9,2012,Australia,AUS,Australia,8,15,12


* Merge with `wb_df_1b` to include GDP, life expectancy and total population to form the `grand_df`

In [52]:
grand_df = match_sorted.merge(
    wb_df_1b[['country', 'GDP (current US$)', 'Life expectancy at birth, total (years)', 'Population, total']],
    left_on='wb_country',
    right_on='country',
    how='left'
)

grand_df.drop(columns=['country'], inplace=True)

grand_df

,year,medals_country,medals_country_code,wb_country,gold,silver,bronze,GDP (current US$),"Life expectancy at birth, total (years)","Population, total"
0,2012,United States,USA,United States,46,28,30,1.625397e+13,78.741463,3.138777e+08
1,2012,People's Republic of China,CHN,China,38,31,22,8.532185e+12,76.192000,1.354190e+09
2,2012,Great Britain,GBR,United Kingdom,29,17,19,2.707090e+12,80.904878,6.370022e+07
3,2012,Russian Federation,RUS,Russian Federation,20,20,27,2.208294e+12,70.072195,1.433784e+08
4,2012,Republic of Korea,KOR,"Korea, Rep.",13,9,8,1.278047e+12,80.819512,5.019985e+07
5,2012,Germany,GER,Germany,11,20,13,3.527143e+12,80.539024,8.042582e+07
6,2012,France,FRA,France,11,11,13,2.683672e+12,81.968293,6.566224e+07
7,2012,Hungary,HUN,Hungary,8,4,6,1.288143e+11,75.063415,9.920362e+06
8,2012,Italy,ITA,Italy,8,9,11,2.086958e+12,82.239024,5.953972e+07
9,2012,Australia,AUS,Australia,8,15,12,1.547650e+12,82.046341,2.273346e+07


In [53]:
pd.reset_option('display.max_rows')  #... resetting the display option

## 2.b Medals per 10 million inhabitants

In [54]:
# Create a new DataFrame for medals per 10 million from the 'grand_df'
medals_10M_df = grand_df[['year', 'medals_country', 'medals_country_code', 'gold', 'silver', 'bronze', 'Population, total']].copy()

# Calculate medals per 10 million inhabitants
medals_10M_df['Gold_per_10M'] = (medals_10M_df['gold'] / (medals_10M_df['Population, total'] / 10000000))
medals_10M_df['Silver_per_10M'] = (medals_10M_df['silver'] / (medals_10M_df['Population, total'] / 10000000))
medals_10M_df['Bronze_per_10M'] = (medals_10M_df['bronze'] / (medals_10M_df['Population, total'] / 10000000))

# Sort by 'Gold_per_10M'
medals_10M_df.sort_values(by='Gold_per_10M', ascending=False, inplace=True)

# Reset the index
medals_10M_df.reset_index(drop=True, inplace=True)

# Display the top 10 successful countries
top_10 = medals_10M_df[['year', 'medals_country', 'medals_country_code', 'Gold_per_10M', 'Silver_per_10M', 'Bronze_per_10M']].head(10)

top_10

,year,medals_country,medals_country_code,Gold_per_10M,Silver_per_10M,Bronze_per_10M
0,2012,Grenada,GRN,86.272345,0.000000,0.000000
1,2012,The Bahamas,BAH,26.173831,0.000000,0.000000
2,2012,Jamaica,JAM,14.493715,18.117143,10.870286
3,2012,New Zealand,NZL,13.611306,4.537102,11.342755
4,2012,Bahrain,BRN,8.163672,0.000000,0.000000
5,2012,Hungary,HUN,8.064222,4.032111,6.048166
6,2012,Croatia,CRO,7.029781,2.343260,4.686521
7,2012,Trinidad and Tobago,TTO,6.991164,6.991164,13.982328
8,2012,Lithuania,LTU,6.693949,0.000000,10.040923
9,2012,Latvia,LAT,4.915650,0.000000,4.915650


# Ex. 3 Supervised Machine Learning

In [55]:
# Display the grand_df
grand_df

,year,medals_country,medals_country_code,wb_country,gold,silver,bronze,GDP (current US$),"Life expectancy at birth, total (years)","Population, total"
0,2012,United States,USA,United States,46,28,30,1.625397e+13,78.741463,3.138777e+08
1,2012,People's Republic of China,CHN,China,38,31,22,8.532185e+12,76.192000,1.354190e+09
2,2012,Great Britain,GBR,United Kingdom,29,17,19,2.707090e+12,80.904878,6.370022e+07
3,2012,Russian Federation,RUS,Russian Federation,20,20,27,2.208294e+12,70.072195,1.433784e+08
4,2012,Republic of Korea,KOR,"Korea, Rep.",13,9,8,1.278047e+12,80.819512,5.019985e+07
...,...,...,...,...,...,...,...,...,...,...
81,2012,Kingdom of Saudi Arabia,KSA,Saudi Arabia,0,0,1,7.418499e+11,76.461000,3.082154e+07
82,2012,Afghanistan,AFG,Afghanistan,0,0,1,1.990733e+10,61.923000,3.046648e+07
83,2012,Slovakia,SVK,Slovak Republic,0,1,3,9.462373e+10,76.109756,5.407579e+06
84,2012,"Hong Kong, China",HKG,"Hong Kong SAR, China",0,0,1,2.626289e+11,83.450244,7.150100e+06


In [56]:
# Create a copy of grand_df for exercise 3 and add a total medals column
grand_df_total = grand_df.copy()
grand_df_total['total_medals'] = grand_df_total['gold'] + grand_df_total['silver'] + grand_df_total['bronze']
total_medals_col = grand_df_total.pop('total_medals')  #... popping total_medals
grand_df_total.insert(7, 'total_medals', total_medals_col)  #... inserting total_medals at the desired position for better readability
grand_df_total.head(10)

,year,medals_country,medals_country_code,wb_country,gold,silver,bronze,total_medals,GDP (current US$),"Life expectancy at birth, total (years)","Population, total"
0,2012,United States,USA,United States,46,28,30,104,1.625397e+13,78.741463,3.138777e+08
1,2012,People's Republic of China,CHN,China,38,31,22,91,8.532185e+12,76.192000,1.354190e+09
2,2012,Great Britain,GBR,United Kingdom,29,17,19,65,2.707090e+12,80.904878,6.370022e+07
3,2012,Russian Federation,RUS,Russian Federation,20,20,27,67,2.208294e+12,70.072195,1.433784e+08
4,2012,Republic of Korea,KOR,"Korea, Rep.",13,9,8,30,1.278047e+12,80.819512,5.019985e+07
5,2012,Germany,GER,Germany,11,20,13,44,3.527143e+12,80.539024,8.042582e+07
6,2012,France,FRA,France,11,11,13,35,2.683672e+12,81.968293,6.566224e+07
7,2012,Hungary,HUN,Hungary,8,4,6,18,1.288143e+11,75.063415,9.920362e+06
8,2012,Italy,ITA,Italy,8,9,11,28,2.086958e+12,82.239024,5.953972e+07
9,2012,Australia,AUS,Australia,8,15,12,35,1.547650e+12,82.046341,2.273346e+07


## 3.a Train and evaluate a linear regression model

### 3.a.1 Split the data into a training and a test set

In [57]:
# Define features
grand_df_total = grand_df_total.dropna()  #... dropping missing values to facilitate the regression algorithm
features = grand_df_total[['GDP (current US$)', 'Life expectancy at birth, total (years)', 'Population, total']]

# Define targets
target_gold = grand_df_total['gold']
target_silver = grand_df_total['silver']
target_bronze = grand_df_total['bronze']
target_total = grand_df_total['total_medals']

# Split the data into training and test set (80:20)

# for gold
X_train_gold, X_test_gold, y_train_gold, y_test_gold = train_test_split(features, target_gold, test_size=0.2, random_state=42)

# for silver
X_train_silver, X_test_silver, y_train_silver, y_test_silver = train_test_split(features, target_silver, test_size=0.2, random_state=42)

# for bronze
X_train_bronze, X_test_bronze, y_train_bronze, y_test_bronze = train_test_split(features, target_bronze, test_size=0.2, random_state=42)

# for total
X_train_total, X_test_total, y_train_total, y_test_total = train_test_split(features, target_total, test_size=0.2, random_state=42)

### 3.a.2 Train the linear regression models

In [58]:
# Initialize the models
model_gold = LinearRegression()
model_silver = LinearRegression()
model_bronze = LinearRegression()
model_total = LinearRegression()

# Train the models
model_gold.fit(X_train_gold, y_train_gold)
model_silver.fit(X_train_silver, y_train_silver)
model_bronze.fit(X_train_bronze, y_train_bronze)
model_total.fit(X_train_total, y_train_total)

# Predict on the test sets
y_pred_gold = model_gold.predict(X_test_gold)
y_pred_silver = model_silver.predict(X_test_silver)
y_pred_bronze = model_bronze.predict(X_test_bronze)
y_pred_total = model_total.predict(X_test_total)

### 3.a.3 Evaluate the model using RMSE

In [29]:
rmse_gold = np.sqrt(mean_squared_error(y_test_gold, y_pred_gold))
rmse_silver = np.sqrt(mean_squared_error(y_test_silver, y_pred_silver))
rmse_bronze = np.sqrt(mean_squared_error(y_test_bronze, y_pred_bronze))
rmse_total = np.sqrt(mean_squared_error(y_test_total, y_pred_total))

print('Gold RMSE:', rmse_gold)
print('Silver RMSE:', rmse_silver)
print('Bronze RMSE:', rmse_bronze)
print('Total RMSE:', rmse_total)

Gold RMSE: 9.569560201018747
Silver RMSE: 15.107941101906865
Bronze RMSE: 14.79212834254983
Total RMSE: 38.43121593302594


## 3.b Briefly discuss the results

1. Judging the model's performance:
* The Gold RMSE of nearly 9.57 indicates that the model's gold medal predictions have an error of about 9.57 medals.

* Similarly, for the no. of Silver, Bronze and Total medals, the RMSE values indicate and error of about 15.11, 14.79 and 38.43 respectively in the model's medal predictions.

* Generally, the lower the RMSE value the better. Thus the model is relatively better at predicting the no. of gold medals than it is at predicting the others.

2. Possible reasons for the performance:
* The relatively small size of the dataset might not be enough to sufficiently train the model.

* The dataset has a lot values where countries have not won any medals which might skew the predictions.

* The features defined to be used with the model might not encapsulate the factors influencing the no. of medals won.

* Linear regression might not be the best method to evaluate this type of dataset.

3. Possible improvements:
* A larger dataset spanning over more years might help.

* Expanding the range of features to more relavant features like the sports budget of the country, historical performance etc.

* Trying out other methods like Decision Tree etc.

## 3.c Predict the number of medals a hypothetical country with a population of 10 million, life expectancy of 70 years, and a GDP per capita of 20.000 US$ would win.

In [59]:
# Denote the hypothetical country as country_x
country_x = pd.DataFrame({
    'GDP (current US$)': [20000],
    'Life expectancy at birth, total (years)': [70],
    'Population, total': [10000000]  
})

# Predict the medal count
predict_gold = model_gold.predict(country_x)
predict_silver = model_silver.predict(country_x)
predict_bronze = model_bronze.predict(country_x)
predict_total = model_total.predict(country_x)

# Print the results
print('Predicted gold medals:', predict_gold[0])
print('Predicted silver medals:', predict_silver[0])
print('Predicted bronze medals:', predict_bronze[0])
print('Predicted total medals:', predict_total[0])

Predicted gold medals: 0.7002618208631892
Predicted silver medals: 0.8397136778318925
Predicted bronze medals: 1.8075213855563863
Predicted total medals: 3.347496883649697


* A.: According to this model, a hypothetical country with a GDP per capita of US$20,000, life expectancy of 70 years at birth and a population of 10 million would win apporximately 0.7 gold medals, 0.84 silver medals, 1.81 bronze medals and a total of 3.35 medals.